In [1]:
from prepare import *


In [2]:
datadf=pd.read_json('data.json')

len(datadf)

# df_C=datadf[datadf['language']=='C'];df_C.head(n=30)
# df_Python=datadf[datadf['language']=='Python'];df_Python.head(n=30)
# df_Java=datadf[datadf['language']=='Java'];df_Java.head(n=30)

all_articles=(datadf['readme_contents']).to_dict()



In [3]:
newdf=dictlist_super_NLP_comp(all_articles)


In [4]:
total=list(pd.read_pickle('words.pkl'))


In [5]:
#some merging gymnastics
a=pd.DataFrame(newdf[0]).T
for i in range(1,len(newdf)):
    b=pd.DataFrame(newdf[i]).T
    a=pd.merge(a,b,how='outer',right_index=False,left_index=False)


    


In [6]:
df=pd.merge(datadf,a,how='left',right_index=True,left_index=True)
display(df.head(),datadf.head())## Verify merge
datadf=df #finish prepped


,repo,language,readme_contents,cleaned,lemmatized,org,stemmed
0,cstack/db_tutorial,C,# Let's Build a Simple Database\n\n[View rende...,let's build a simple database\n\n view rende...,let build simple view tutorial io tutorial run...,# Let's Build a Simple Database\n\n[View rende...,let build simple view tutorial io tutorial run...
1,rui314/chibicc,C,# chibicc: A Small C Compiler\n\n(The old mast...,chibicc a small c compiler\n\n the old mast...,small compiler old master old tree old branch ...,# chibicc: A Small C Compiler\n\n(The old mast...,small compiler old master old tree old branch ...
2,nelhage/reptyr,C,"reptyr - A tool for ""re-ptying"" programs.\n===...",reptyr a tool for re ptying programs \n ...,tool utility taking running program new termin...,"reptyr - A tool for ""re-ptying"" programs.\n===...",tool utility taking running program new termin...
3,EyalAr/lwip,C,[![Version](http://img.shields.io/npm/v/lwip.s...,version http img shields io npm v lwip s...,version io package build status travis branch ...,[![Version](http://img.shields.io/npm/v/lwip.s...,version io package build status travis branch ...
4,ibireme/yyjson,C,\n# Introduction\n\n[![Build](https://flat.bad...,\n introduction\n\n build https flat bad...,build flat net status master flat net io licen...,\n# Introduction\n\n[![Build](https://flat.bad...,build flat net status master flat net io licen...


,repo,language,readme_contents
0,cstack/db_tutorial,C,# Let's Build a Simple Database\n\n[View rende...
1,rui314/chibicc,C,# chibicc: A Small C Compiler\n\n(The old mast...
2,nelhage/reptyr,C,"reptyr - A tool for ""re-ptying"" programs.\n===..."
3,EyalAr/lwip,C,[![Version](http://img.shields.io/npm/v/lwip.s...
4,ibireme/yyjson,C,\n# Introduction\n\n[![Build](https://flat.bad...


In [7]:
reorder=['language',
'repo',
 'readme_contents',
 'cleaned',
 'stemmed',
 'lemmatized']

datadf=datadf[reorder] # reordered the columns 

pd.to_pickle(datadf,'cleanpickle.pkl')# saved to pickle for later